# K近邻算法
+ 既可以是分类（类别），也可以是回归（具体值）

### Kmeans（K均值）与KNN（K近邻）的区别
Kmeans是把数据分成K类，KNN是把K个相邻近的数据归为一组
+ Kmeans它把n个对象根据他们的属性分为k个聚类以便使得所获得的聚类满足：同一聚类中的对象相似度较高；而不同聚类中的对象相似度较小
+ KNN如果一个样本在特征空间中的k个最相似(即特征空间中最邻近)的样本中的大多数属于某一个类别，则该样本也属于这个类别。该方法在定类决策上只依据最邻近的一个或者几个样本的类别来决定待分样本所属的类别

### 数据结构--数据框dataframe

In [ ]:
import numpy as np
import pandas as pd
from pandas import Sereis, DataFrame
ser = Series(np.arange(3.))
data = DataFrame(np.arange(16).reshape(4,4),index=list('abcd'),columns=list('wxyz'))
data['w']  #选择表格中的'w'列，使用类字典属性,返回的是Series类型
data.w    #选择表格中的'w'列，使用点属性,返回的是Series类型
data[['w']]  #选择表格中的'w'列，返回的是DataFrame类型
data[['w','z']]  #选择表格中的'w'、'z'列
data[0:2]  #返回第1行到第2行的所有行，前闭后开，包括前不包括后
data[1:2]  #返回第2行，从0计，返回的是单行，通过有前后值的索引形式 如果采用data[1]则报错
data.ix[1:2] #返回第2行的第三种方法，返回的是DataFrame，跟data[1:2]同
data['a':'b']  #利用index值进行切片，返回的是**前闭后闭**的DataFrame 即末端是包含的  
data.irow(0)   #取data的第一行
data.icol(0)   #取data的第一列
data.head()  #返回data的前几行数据，默认为前五行，需要前十行则data.head(10)
data.tail()  #返回data的后几行数据，默认为后五行，需要后十行则data.tail(10)
ser.iget_value(0)  #选取ser序列中的第一个
ser.iget_value(-1) #选取ser序列中的最后一个，这种轴索引包含索引器的series不能采用ser[-1]去获取最后一个，这会引起歧义。
data.iloc[-1]   #选取DataFrame最后一行，返回的是Series
data.iloc[-1:]   #选取DataFrame最后一行，返回的是DataFrame
data.loc['a',['w','x']]   #返回‘a’行'w'、'x'列，这种用于选取行索引列索引已知
data.iat[1,1]   #选取第二行第二列，用于已知行、列位置的选取。

### Pandas库中的sample--随机抽样
DataFrame.sample(n=None, frac=None, replace=False, weights=None, random_state=None, axis=None)
+ n：要抽取的行数
+ frac：抽取行的比例，例如frac=0.8，就是抽取其中80%
+ replace：是否为有放回抽样，True:有放回抽样，False:未放回抽样
+ weights：字符索引或概率数组，xis=0:为行字符索引或概率数组，axis=1:为列字符索引或概率数组
+ random_state：int: 随机数发生器种子，或numpy.random.RandomState。random_state=None,取得数据不重复，random_state=1,可以取得重复数据
+ axis：选择抽取数据的行还是列，axis=0:抽取行，axis=1:抽取列

### 替换replace
str.replace(old, new[, max]) 

参数解读
+ old -- 将被替换的子字符串。
+ new -- 新字符串，用于替换old子字符串。
+ max -- 可选字符串, 替换不超过 max 次
 
替换字符含义：
+ ^ 表示以什么开头
+ \s 代表匹配一些空白符     包括：空格、Tab、换行、回车,等价于 [\t\r\n ]
+ *表示有一个或者多个
+ | 表示或者
+ $ 代表以什么结尾

In [4]:
import pandas as pd

# 选取数据的字段
features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews'] 

# 读取数据
dc_listings = pd.read_csv(r'D:/001  学习文件/001  学习笔记/020  唐宇迪机器学习实战_20200326/机器学习文件/006 K近邻实例/KNN/listings.csv') 

# 数据框取值
dc_listings = dc_listings[features] 
# 数据行列数
print(dc_listings.shape) 

# 查看对象的前n行数据
dc_listings.head()

(3723, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,$160.00,1,1125,0
1,6,3.0,3.0,3.0,$350.00,2,30,65
2,1,1.0,2.0,1.0,$50.00,2,1125,1
3,2,1.0,1.0,1.0,$95.00,1,1125,0
4,4,1.0,1.0,1.0,$50.00,7,1125,0


#### 数据特征：

* accommodates: 可以容纳的旅客
* bedrooms: 卧室的数量
* bathrooms: 厕所的数量
* beds: 床的数量
* price: 每晚的费用
* minimum_nights: 客人最少租了几天
* maximum_nights: 客人最多租了几天
* number_of_reviews: 评论的数量

## K近邻原理

### 如果我有一个3个房间的房子，我能租多少钱呢？
讲道理，咱是不是得看一看3个房间的别人都租到多少钱啊？--货比三家

K代表我们的候选对象个数，也就是我和我房间数量最相近的其他房子的价格

### 疑问：为什么取的是最大的3个？？？--似乎是随机选取

## 距离的定义
如何让才能知道哪些数据样本跟我最相近呢？   
通过[欧式距离](https://baike.baidu.com/item/%E6%AC%A7%E5%87%A0%E9%87%8C%E5%BE%97%E5%BA%A6%E9%87%8F/1274107?fromtitle=%E6%AC%A7%E5%BC%8F%E8%B7%9D%E7%A6%BB&fromid=2809635&fr=aladdin) 进行计算，详细公式如下： 

![欧式距离](http://note.youdao.com/yws/public/resource/882a672748c498993685201f96a6c217/xmlnote/C877D0753C444631AA9A7E8591A12824/23542)

其中Q1到Qn是一条数据的所有特征信息，P1到Pn是另一条数据的所有特征信息

假设我的房间数量是3个：

In [9]:
import numpy as np

our_acc_value = 3

# 用所有数据房间数减去3，然后结果取绝对值
dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)

# 按差值分组统计，并排序
dc_listings.distance.value_counts().sort_index()

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

这里我们只用了绝对值来计算，和我们距离为0的（同样数量的房间）有461个

#### sample操作可以得到洗牌后数据--随机抽样

In [14]:
# 随机打乱数据
dc_listings = dc_listings.sample(frac=1,random_state=0) 

# 按距离排序
dc_listings = dc_listings.sort_values('distance')

# 显示前5行
dc_listings.price.head()

2811    $120.00
408      $95.00
316     $155.00
1040    $299.00
1682    $165.00
Name: price, dtype: object

需要注意的是，这里面的数据现在是字符串，如果想进行计算，需要转换一下。

In [15]:
# 替换“$”或者，为空
dc_listings['price'] = dc_listings.price.str.replace("\$|,","").astype(float)

# 取前5的数据的平均值
mean_price = dc_listings.price.iloc[:5].mean()

mean_price

166.8

得到了平均价格，也就是我们房子的大致价格。

## 模型评估

![模型评估](http://note.youdao.com/yws/public/resource/882a672748c498993685201f96a6c217/xmlnote/3BE562C6F1F54727B4896152CE902873/23546)

首先制定好训练集和测试集，训练集训练模型，出现的模型尽量贴合训练数据；测试集是新的数据，来衡量模型的结果